In [1]:
import abc, fnmatch, IPython, functools

In [2]:
import vdom; from vdom import div, img; from vdom.svg import iframe
from mimetypes import guess_type; guess = lambda x: guess_type(x)[0]

In [3]:
    class StringDisplayBase(abc.ABCMeta):
        def append(self, key):
            def attach(callable): 
                setattr(self, key, callable)
                return callable
            return attach

        def __call__(self, str, result=None):
            for attr in (x for x in dir(self) if '*' in x):
                if fnmatch.fnmatch(str, attr):
                    result = getattr(self, attr)(str)
                    
                    if isinstance(result, dict) and str in result:
                        result = result[str]
                        if result['status'] == 'ok':
                            return result['data'], result['metadata']
                        else: ...

                    elif result is not None:
                        data, metadata = IPython.get_ipython().display_formatter.format(result)
                        return data, metadata
                    
                
            if str and str.splitlines()[0].strip():
                return {
                    'text/markdown': str
                }, {}
    class StringDisplay(metaclass=StringDisplayBase): ...

In [4]:
import IPython

In [5]:
@StringDisplay.append('graph*{*}')
@StringDisplay.append('digraph*{*}')
def graphviz(str):
    return __import__('graphviz').Source(str)

In [6]:
from .flexlist import Row

In [15]:
@StringDisplay.append('- *')
@StringDisplay.append('[[]*[]]')
def yaml(str):
    import yaml, io
    data = yaml.safe_load(io.StringIO(str))
    if str.startswith('[') and len(data) == 1:
        result = imports(str[1:-1])
        if result: return result
    return Row(data)

In [16]:
@StringDisplay.append('http*')
def embed(str):
    type = guess(str) or ''
    if str.startswith('http'): 
        if type.startswith('image') and not type.endswith('svg'):
            return img(src=str)
        return iframe(src=str, style=dict(width='100%', height="400px"))

In [17]:
import sys

In [18]:
@StringDisplay.append('*')
def imports(str):
    if str[0] == str[1] == '`': return
    try:
        return __import__(str)
    except:...
        
    if hasattr(__import__('__main__'), str):
        return getattr(__import__('__main__'), str)
    
    if str in sys.modules:
        return sys.modules[str] 

In [19]:
def load_ipython_extension(ip):
    ip.display_formatter.mimebundle_formatter.for_type(str, StringDisplay)
    
def unload_ipython_extension(ip):
    ip.display_formatter.mimebundle_formatter.for_type(str)
    
__name__ == '__main__' and load_ipython_extension(get_ipython())